In [9]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import time
import cv2
from tqdm.notebook import tqdm
from collections import Counter
import unicodedata
import ast
import json

In [1]:
import zipfile
import os

zip_path = "/content/vietnamese.zip"
extract_path = "/content/vietnamese"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Đã giải nén VinText.")

Đã giải nén VinText.


In [4]:
import os

label_dir = '/content/vietnamese/vietnamese/labels'
img_dir = '/content/vietnamese/vietnamese/train_images'

fullimg_infos = []
for label_file in sorted(os.listdir(label_dir)):
    # Kiểm tra chỉ lấy file label dạng gt_*.txt
    if not label_file.startswith('gt_') or not label_file.endswith('.txt'):
        continue
    # Lấy số thứ tự ảnh
    num_str = label_file.replace('gt_', '').replace('.txt', '')
    img_file = f'im{int(num_str):04d}.jpg'   # im0001.jpg
    img_path = os.path.join(img_dir, img_file)
    if not os.path.exists(img_path):
        continue
    label_path = os.path.join(label_dir, label_file)
    with open(label_path, encoding='utf-8') as f:
        lines = f.readlines()
    transcripts = []
    for line in lines:
        items = line.strip().split(',', 8)
        if len(items) < 9: continue
        transcript = items[8]
        if transcript == "###": continue
        transcripts.append(transcript)
    joint_text = " ".join(transcripts)
    fullimg_infos.append((img_file, joint_text))

# Lưu lại label cho train
with open('/content/vietnamese/vietnamese/train_fullimg_label.txt', 'w', encoding='utf-8') as f:
    for img_file, joint_text in fullimg_infos:
        f.write(f"{img_file}\t{joint_text}\n")

print(f"Đã tạo ground_truth cho {len(fullimg_infos)} ảnh train.")

Đã tạo ground_truth cho 1200 ảnh train.


In [5]:
# Tạo vocab từ file ground_truth đã tạo ở bước trước
label_txt = '/content/vietnamese/vietnamese/train_fullimg_label.txt'
vocab = set()
with open(label_txt, encoding='utf-8') as f:
    for line in f:
        parts = line.strip().split('\t')
        if len(parts) == 2:
            _, text = parts
            vocab.update(list(text))

special_tokens = ['<pad>', '<sos>', '<eos>', '<unk>']
vocab = special_tokens + sorted(vocab)
stoi = {ch: i for i, ch in enumerate(vocab)}
itos = {i: ch for ch, i in stoi.items()}

def encode(text):
    return [stoi['<sos>']] + [stoi.get(ch, stoi['<unk>']) for ch in text] + [stoi['<eos>']]

def decode(tokens):
    return ''.join([itos[t] for t in tokens if t not in [stoi['<sos>'], stoi['<eos>'], stoi['<pad>']]])

print(f"Số lượng ký tự trong vocab (kể cả token đặc biệt): {len(vocab)}")
print("Vài ký tự đầu:", vocab[:10])

Số lượng ký tự trong vocab (kể cả token đặc biệt): 230
Vài ký tự đầu: ['<pad>', '<sos>', '<eos>', '<unk>', ' ', '!', '"', '#', '$', '%']


In [6]:
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

img_dir = '/content/vietnamese/vietnamese/train_images'
label_txt = '/content/vietnamese/vietnamese/train_fullimg_label.txt'

transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB')),
    transforms.Resize((64, 256)),  # có thể tùy chỉnh theo ảnh thực tế
    transforms.ToTensor(),
])

class OCRFullImgDataset(Dataset):
    def __init__(self, img_dir, label_txt, transform):
        self.img_dir = img_dir
        self.transform = transform
        self.samples = []
        with open(label_txt, encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) == 2:
                    fname, label = parts
                    self.samples.append((fname, label))
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        fname, label = self.samples[idx]
        img = Image.open(os.path.join(self.img_dir, fname))
        img = self.transform(img)
        label_ids = encode(label)
        return img, torch.tensor(label_ids, dtype=torch.long)

def collate_fn(batch):
    imgs, labels = zip(*batch)
    imgs = torch.stack(imgs)
    maxlen = max(len(l) for l in labels)
    labels_pad = [torch.cat([l, torch.full((maxlen - len(l),), stoi['<pad>'], dtype=torch.long)]) for l in labels]
    labels_pad = torch.stack(labels_pad)
    return imgs, labels_pad

train_dataset = OCRFullImgDataset(img_dir, label_txt, transform)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

print("Số batch:", len(train_loader))
imgs, labels = next(iter(train_loader))
print("Shape ảnh batch:", imgs.shape)
print("Shape label batch:", labels.shape)

Số batch: 150
Shape ảnh batch: torch.Size([8, 3, 64, 256])
Shape label batch: torch.Size([8, 177])


In [19]:
import torch
import torch.nn as nn
from torchvision.models import resnet18
import math

class CNNEncoder(nn.Module):
    def __init__(self, out_dim):
        super().__init__()
        base = resnet18(weights=None)
        modules = list(base.children())[:-2]
        self.cnn = nn.Sequential(*modules)
        self.conv_proj = nn.Conv2d(512, out_dim, 1)

    def forward(self, x):
        x = self.cnn(x)
        x = self.conv_proj(x)
        B, C, H, W = x.shape
        # print(f"Feature shape after CNN: {x.shape}, seq_len={H*W}")  # Xem seq_len thực tế
        x = x.flatten(2).permute(2, 0, 1)
        return x

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=2000):  # Đặt max_len lớn
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(1))

    def forward(self, x):
        x = x + self.pe[:x.size(0)].to(x.device)
        return x

class Seq2SeqOCR(nn.Module):
    def __init__(self, vocab_size, d_model=128, num_layers=2, dropout=0.1, max_len=2000):
        super().__init__()
        self.encoder = CNNEncoder(d_model)
        self.decoder_layer = nn.TransformerDecoderLayer(d_model, nhead=4, dropout=dropout)
        self.decoder = nn.TransformerDecoder(self.decoder_layer, num_layers)
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, max_len)
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, imgs, tgt_input, tgt_mask=None):
        memory = self.encoder(imgs)
        tgt_emb = self.embedding(tgt_input).permute(1,0,2)
        tgt_emb = self.pos_encoder(tgt_emb)
        out = self.decoder(tgt_emb, memory, tgt_mask=tgt_mask)
        out = self.fc_out(out)
        return out

print('Đã sửa model với max_len=2000! Sẵn sàng huấn luyện.')

Đã sửa model với max_len=2000! Sẵn sàng huấn luyện.


In [21]:
import torch
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Seq2SeqOCR(len(vocab)).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

EPOCHS = 40
print_freq = 20  # Số batch sẽ in log chi tiết 1 lần
best_loss = float('inf')
save_path = 'best_ocr_model.pth'

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    total_tokens = 0
    total_correct = 0
    print(f"\n========== EPOCH {epoch+1} ==========")
    for batch_idx, (imgs, labels) in enumerate(tqdm(train_loader)):
        imgs, labels = imgs.to(device), labels.to(device)
        tgt_input = labels[:, :-1]
        tgt_output = labels[:, 1:]
        tgt_mask = nn.Transformer.generate_square_subsequent_mask(tgt_input.size(1)).to(device)
        out = model(imgs, tgt_input, tgt_mask=tgt_mask)
        out = out.permute(1, 0, 2)  # [batch, seq, vocab]
        loss = F.cross_entropy(out.reshape(-1, len(vocab)), tgt_output.reshape(-1), ignore_index=stoi['<pad>'])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        # Thống kê char accuracy trên batch
        preds = out.argmax(dim=-1)  # [batch, seq]
        mask = tgt_output != stoi['<pad>']
        correct = ((preds == tgt_output) & mask).sum().item()
        n_tokens = mask.sum().item()
        total_tokens += n_tokens
        total_correct += correct

        if (batch_idx + 1) % print_freq == 0 or (batch_idx == 0):
            batch_acc = correct / (n_tokens + 1e-9)
            print(f"[Epoch {epoch+1} | Batch {batch_idx+1}/{len(train_loader)}] "
                  f"Loss: {loss.item():.4f} | CharAcc: {batch_acc:.3f} | "
                  f"Batch size: {imgs.size(0)} | Seq len: {tgt_input.size(1)}")

            # Hiển thị ví dụ 1 sample đầu tiên trong batch (decode chuỗi)
            sample_pred = decode(preds[0].tolist())
            sample_gt = decode(tgt_output[0].tolist())
            print(f"  [Sample GT]:   {sample_gt}")
            print(f"  [Sample Pred]: {sample_pred}")

    avg_loss = total_loss / len(train_loader)
    avg_acc = total_correct / (total_tokens + 1e-9)
    print(f"\n>>> Epoch {epoch+1} completed. Avg Loss: {avg_loss:.4f} | Avg CharAcc: {avg_acc:.4f}")

    # Lưu model tốt nhất
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
            'vocab': vocab,
            'stoi': stoi,
            'itos': itos
        }, save_path)
        print(f"==> Saved new best model at epoch {epoch+1} (loss={best_loss:.4f})")

print("Huấn luyện hoàn tất. Model tốt nhất đã được lưu tại:", save_path)


========== EPOCH 1 ==========


  1%|▏         | 2/150 [00:00<00:34,  4.26it/s]

[Epoch 1 | Batch 1/150] Loss: 5.6402 | CharAcc: 0.001 | Batch size: 8 | Seq len: 192
  [Sample GT]:   VNSTEEL THANG LONG KIẾN TẠO GIÁ TRỊ CÔNG TRÌNH TÔN VNSTEEL THĂNG LONG Lô 14, Khu công nghiệp Quang Minh, Mê Linh, Hà Nội K P www. vnsteelthanglong.vn
  [Sample Pred]: fẬỸÈỄỄậÌạỄGỄỳỢÌỢỄèạ4Ợ<bFỄ2ỢạỒÌFFrbỄửY$ỄỒửỄử&ửửFử2ỄạửEÈỄỄ<ÌửỄửưỄụạ>ÃỄèFÌỄFỄỄFạb>ửFỄèỄ`FỄ`Ýế>cạồÝỒÝèyÌ2ỄỷYạÌ<ạ>2oạ6ạbỄạỄỄ2>oF>yFFF<<ỒỬỂFỒỳỸửỷỒỬ<<bỬ<<ỒỄỬ<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


 14%|█▍        | 21/150 [00:05<00:23,  5.40it/s]

[Epoch 1 | Batch 20/150] Loss: 3.7029 | CharAcc: 0.227 | Batch size: 8 | Seq len: 182
  [Sample GT]:   CỬA CUỐN CỬA KÍNH QUANG VINH HOTLINE: 0313582228 0934344228 QV
  [Sample Pred]: N   T    T   T    T     T    T        T          T          T                                                                                                                         


...

[Epoch 40 | Batch 120/150] Loss: 1.2368 | CharAcc: 0.623 | Batch size: 8 | Seq len: 201
  [Sample GT]:   PHÓ BÍ THƯ THƯỜNG TRỰC HUYỆN UỶ
  [Sample Pred]: CHỞ B  THẨỜHHƯỜNG BHUC BUYÊN TỶ  COOCCO  O CCCCCC T A 4  SCCC4U644C CCTCCCC CCTCMTCOTọTCC  C CCCC4 CCCCCC CCMCSC CC  C CCCCO4    CP  6S4CTC 4CC6CCDOC44C 44444K40 0CC C 44CCC  4CCCC4 CSCCCTOC CMCC  


 93%|█████████▎| 140/150 [00:42<00:02,  3.41it/s]

[Epoch 40 | Batch 140/150] Loss: 1.2887 | CharAcc: 0.606 | Batch size: 8 | Seq len: 234
  [Sample GT]:   AVIVA LỄ KHAI TRƯƠNG VĂN PHÒNG KINH DOANH AVIVA VIỆT NAM Đồng Thap1, ngày 1 /2019 VẠN SỰ NHƯ Ý KHAI TRƯƠNG HỒNG MÃ ĐÁO THÀNH CÔNG
  [Sample Pred]: CNI A VỄ KHÁI TRƯƠNG BÕN KHÍNG VIỆH KAAN  NNI A PIỆT LAM VI g Chịn1, tgày 1 k1019 NyN BÚ LăƯ Ý KHÁN TRƯƠNG HỒNG KỸ HẠO HẠÀNH ChNG  NNCN 6N NC6)CỞƠ) C  Ở  4DB4OS4K K 0C NÔ  C4DC  Ú CCCC  ƠCAỞƠ  C0M4DPC44 S4 C     C) 44 C 0D4CC)K C    


100%|██████████| 150/150 [00:44<00:00,  3.37it/s]



>>> Epoch 40 completed. Avg Loss: 1.1624 | Avg CharAcc: 0.6435
==> Saved new best model at epoch 40 (loss=1.1624)
Huấn luyện hoàn tất. Model tốt nhất đã được lưu tại: best_ocr_model.pth
